In [1]:
%load_ext autoreload
%autoreload 2

import sys
import time
import random
import torch

import numpy as np
import torch.nn as nn
import torch.optim as optim

from src.model import *
from src.load_data import load_data

from pathlib import Path
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors

In [2]:
SEED = 1234
MIN_FREQ = 1
BATCH_SIZE = 64

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
windows = [2, 5, 10]
vector_sizes = [0, 50, 100, 500]
languages = ['bxr']#['bxr', 'myv', 'kv']#['bxr', 'myv', 'kv']
methods = ['cbow', 'sg', 'glove']#[-1, 'cbow', 'sg', 'glove', 'pmi']

In [5]:
scores_d = {}
for language in languages:
    data_generator = load_data(language = language, SEED = SEED)
    
    scores_d[language] = {}
    for method in methods:
        if method not in scores_d[language]:
            scores_d[language][method] = {}
        
        for vector_size in vector_sizes:
            if vector_size not in scores_d[language][method]:
                scores_d[language][method][vector_size] = {}
                    
            for window in windows:            
                if window not in scores_d[language][method][vector_size]:
                    scores_d[language][method][vector_size][window] = []
                
                print("language:", language, "method:", method, "vector_size:", vector_size, "window:", window)
                if method != -1:
                    path = Path("./embeddings/"+language+"/"+str(vector_size)+"/"+str(window)+"/"+method)
                    if not path.is_file():
                        print("File", path, "doesn't exist")
                        continue
                    else:
                        print("loaded word embeddings", path)
                else:
                    if vector_size == 0:
                        print("skipping")
                        continue
                
                for NUM, TEXT, LEMMA, UD_TAGS, train_data, val_data in data_generator.get_fold_data():
                    
                    print("train_data", len(train_data.examples), "val_data", len(val_data.examples))
                    
                    if method == -1:
                        TEXT.build_vocab(train_data, min_freq = MIN_FREQ)
                        emb_size = vector_size
                    else:
                        !rm -rf /tmp/vec
                        _vectors = Vectors(name=path, cache='/tmp/vec')
                        TEXT.build_vocab(train_data,
                            min_freq = MIN_FREQ,
                            vectors = _vectors,
                            unk_init = torch.Tensor.normal_
                        )
                        emb_size = TEXT.vocab.vectors[1].shape[0]
                        
                        if vector_size !=0:
                            assert vector_size==TEXT.vocab.vectors[1].shape[0], "Different sizes"
                        
                    LEMMA.build_vocab(train_data)
                    NUM.build_vocab(train_data)
                    UD_TAGS.build_vocab(train_data)
                    
                    print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
                    print(f"Unique tokens in UD_TAG vocabulary: {len(UD_TAGS.vocab)}")
                    
                    train_iterator, val_iterator = data.BucketIterator.splits(
                                                        (train_data, val_data),
                                                        sort=True,
                                                        sort_key=lambda x: len(x.text),
                                                        sort_within_batch=False,
                                                        batch_size=BATCH_SIZE, 
                                                        repeat=False,
                                                        shuffle=True,
                                                        device=device)
                    
                    pos_score = pos(TEXT, UD_TAGS, train_iterator, val_iterator, emb_size, epochs=30, verbose = False)
                    print("score:", pos_score)
                    scores_d[language][method][vector_size][window].append(pos_score)
                scores_d[language][method][vector_size][window] = np.average(scores_d[language][method][vector_size][window])

language: bxr method: cbow vector_size: 0 window: 2
File embeddings/bxr/0/2/cbow doesn't exist
language: bxr method: cbow vector_size: 0 window: 5
File embeddings/bxr/0/5/cbow doesn't exist
language: bxr method: cbow vector_size: 0 window: 10
File embeddings/bxr/0/10/cbow doesn't exist
language: bxr method: cbow vector_size: 50 window: 2
loaded word embeddings embeddings/bxr/50/2/cbow
train_data 772 val_data 86


/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
100%|██████████| 6124/6124 [00:00<00:00, 47169.85it/s]
/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning

Unique tokens in TEXT vocabulary: 3655
Unique tokens in UD_TAG vocabulary: 17
The model has 94,353 trainable parameters
load pretrained embs
embeddings torch.Size([3655, 50])


/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


score: 0.2833191737042905
train_data 772 val_data 86


...

Unique tokens in TEXT vocabulary: 3672
Unique tokens in UD_TAG vocabulary: 17
The model has 324,753 trainable parameters
load pretrained embs
embeddings torch.Size([3672, 500])
score: 0.5257485161162305
train_data 773 val_data 85


100%|██████████| 6125/6125 [00:00<00:00, 9452.47it/s]


Unique tokens in TEXT vocabulary: 3666
Unique tokens in UD_TAG vocabulary: 17
The model has 324,753 trainable parameters
load pretrained embs
embeddings torch.Size([3666, 500])
score: 0.46979363020727116
train_data 773 val_data 85


100%|██████████| 6125/6125 [00:00<00:00, 9551.24it/s]


Unique tokens in TEXT vocabulary: 3692
Unique tokens in UD_TAG vocabulary: 17
The model has 324,753 trainable parameters
load pretrained embs
embeddings torch.Size([3692, 500])
score: 0.472614666542615


In [153]:
count = 0
for idx, vec in enumerate(TEXT.vocab.vectors):
    if vec[0] == 0 and not TEXT.vocab.itos[idx].isdigit():
#         print(TEXT.vocab.itos[idx])
        count += 1

print('словарь', len(TEXT.vocab.vectors))
print('не найдено', count)

словарь 3655
не найдено 1329


In [236]:
TEXT.vocab.stoi[TEXT.pad_token]

1

In [4]:
def pos(TEXT, UD_TAGS, train_iterator, val_iterator, emb_size, epochs=10, verbose = True):
      
    INPUT_DIM = len(TEXT.vocab)
    EMBEDDING_DIM = emb_size
    HIDDEN_DIM = 128
    OUTPUT_DIM = len(UD_TAGS.vocab)
    N_LAYERS = 1
    BIDIRECTIONAL = False
    DROPOUT = 0.25
    PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

    model = BiLSTMPOSTagger(INPUT_DIM, 
                        EMBEDDING_DIM, 
                        HIDDEN_DIM, 
                        OUTPUT_DIM, 
                        N_LAYERS, 
                        BIDIRECTIONAL, 
                        DROPOUT,
                        PAD_IDX)
    
    model.apply(init_weights)
    print(f'The model has {count_parameters(model):,} trainable parameters')
    
    if TEXT.vocab.vectors is not None:
        print("load pretrained embs")
        pretrained_embeddings = TEXT.vocab.vectors
        print('embeddings', pretrained_embeddings.shape)
    
        model.embedding.weight.data.copy_(pretrained_embeddings)
        model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
    else:
        print("no loading any pretrained embs")
    
    optimizer = optim.Adam(model.parameters())
    TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]
    criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)
    
    model = model.to(device)
    criterion = criterion.to(device)
    
    for epoch in range(epochs):

        start_time = time.time()
    
        train_loss = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
        valid_loss, valid_acc = evaluate(model, val_iterator, criterion, TAG_PAD_IDX)
    
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
            
        if verbose:
            print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
            print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    test_loss, test_acc = evaluate(model, val_iterator, criterion, TAG_PAD_IDX)
    return test_acc

In [7]:
from src.eval import tabular
tabular(scores_d, 'bxr')

0
& 50  &  0.132 & 0.132 & 0.132 \\
& 100  &  0.205 & 0.205 & 0.205 \\
& 500  &  0.484 & 0.484 & 0.484 \\
cbow
& 50  &  0.299 & 0.298 & 0.322 \\
& 100  &  0.307 & 0.314 & 0.329 \\
& 500  &  0.332 & 0.347 & 0.362 \\
sg
& 50  &  0.325 & 0.32 & 0.327 \\
& 100  &  0.349 & 0.369 & 0.378 \\
& 500  &  0.398 & 0.412 & 0.424 \\
glove
& 50  &  0.361 & 0.362 & 0.371 \\
& 100  &  0.389 & 0.396 & 0.419 \\
& 500  &  0.411 & 0.482 & 0.49 \\
pmi
& 50  &  0.402 & 0.417 & 0.467 \\
& 100  &  0.491 & 0.502 & 0.515 \\
& 500  &  0.56 & 0.564 & 0.569 \\


/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


True

In [ ]:
0
& 50  &  0.132 & 0.132 & 0.132 \\
& 100  &  0.205 & 0.205 & 0.205 \\
& 500  &  0.484 & 0.484 & 0.484 \\
cbow
& 50  &  0.299 & 0.298 & 0.322 \\
& 100  &  0.307 & 0.314 & 0.329 \\
& 500  &  0.332 & 0.347 & 0.362 \\
sg
& 50  &  0.325 & 0.32 & 0.327 \\
& 100  &  0.349 & 0.369 & 0.378 \\
& 500  &  0.398 & 0.412 & 0.424 \\
glove
& 50  &  0.361 & 0.362 & 0.371 \\
& 100  &  0.389 & 0.396 & 0.419 \\
& 500  &  0.411 & 0.482 & 0.49 \\
pmi
& 50  &  0.402 & 0.417 & 0.467 \\
& 100  &  0.491 & 0.502 & 0.515 \\
& 500  &  0.56 & 0.564 & 0.569 \\

In [8]:
from src.eval import tabular
tabular(scores_d, 'myv')

0
& 50  &  0.268 & 0.268 & 0.268 \\
& 100  &  0.353 & 0.353 & 0.353 \\
& 500  &  0.535 & 0.535 & 0.535 \\
cbow
& 50  &  0.351 & 0.368 & 0.386 \\
& 100  &  0.367 & 0.382 & 0.407 \\
& 500  &  0.388 & 0.396 & 0.429 \\
sg
& 50  &  0.392 & 0.388 & 0.377 \\
& 100  &  0.413 & 0.41 & 0.423 \\
& 500  &  0.435 & 0.442 & 0.455 \\
glove
& 50  &  0.415 & 0.426 & 0.425 \\
& 100  &  0.444 & 0.469 & 0.465 \\
& 500  &  0.472 & 0.491 & 0.508 \\
pmi
& 50  &  0.429 & 0.473 & 0.458 \\
& 100  &  0.479 & 0.501 & 0.502 \\
& 500  &  0.539 & 0.541 & 0.55 \\


/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


True

In [9]:
from src.eval import tabular

tabular(scores_d, 'kv')

-1


/home/com/miniconda2/envs/py37/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


TypeError: type list doesn't define __round__ method

In [6]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(scores_d)

{   'bxr': {   'cbow': {   0: {2: [], 5: [], 10: []},
                           50: {   2: 0.2744683291756316,
                                   5: 0.25693164168470434,
                                   10: 0.30907419686316473},
                           100: {   2: 0.28801999365118036,
                                    5: 0.27907984126493013,
                                    10: 0.31124381362186326},
                           500: {   2: 0.31355414355458205,
                                    5: 0.3266307007840131,
                                    10: 0.35063415636628714}},
               'glove': {   0: {2: [], 5: [], 10: []},
                            50: {   2: 0.3485984142974646,
                                    5: 0.3459197563308106,
                                    10: 0.35361080296621866},
                            100: {   2: 0.3808066278368199,
                                     5: 0.38912861715681557,
                                     10: 0.40909

In [13]:
import json
with open('extrinsic.json', 'w', encoding='utf-8') as f:
    json.dump(scores_d, f, sort_keys=False, indent=4, ensure_ascii=False)